In [1]:
import pandas as pd
from tortreinador.utils.tools import check_outlier
from tortreinador.utils.plot import plot_line_2
from tortreinador.utils.preprocessing import load_data
from tortreinador.train import TorchTrainer
from Rock.Model.MLP import MLP_Attention
from Rock.Model.SelfAttention import torch_cov
from tortreinador.utils.View import init_weights, split_weights
from tortreinador.utils.plot import calculate_GMM
import torch
import torch.nn as nn
from tortreinador.utils.metrics import r2_score
import matplotlib
from matplotlib import pyplot as plt
import proplot as pplt
import joblib

In [2]:
df_chunk_0 = pd.read_parquet("D:\\Resource\\rockyExoplanetV3\\data_chunk_0.parquet")
df_chunk_1 = pd.read_parquet("D:\\Resource\\rockyExoplanetV3\\data_chunk_1.parquet")

df_all = pd.concat([df_chunk_0, df_chunk_1])

In [3]:
input_parameters = [
    'Mass',
    'Radius',
    'FeMg',
    'SiMg',
]

output_parameters = [
    'WRF',
    'MRF',
    'CRF',
    'WMF',
    'CMF',
    'CPS',
    'CTP',
    'k2'
]

In [4]:
t_loader, v_loader, test_x, test_y, s_x, s_y = load_data(data=df_all, input_parameters=input_parameters,
                                                         output_parameters=output_parameters,
                                                         if_normal=True, if_shuffle=True, batch_size=256)

In [5]:
model = MLP_Attention(input_size=len(input_parameters), output_size=len(output_parameters), layer1_size=256, layer2_size=128, mode='cov')
init_weights(model)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(split_weights(model), lr=0.0001, weight_decay=0.0001)

In [6]:
class Trainer(TorchTrainer):
    def calculate(self, x, y, mode='t'):
        output, weight = model(x)
        y_pred = torch.mean(output, dim=0)

        y = torch_cov(y).to(torch.float64)
        loss = self.criterion(y_pred, y)

        metric_per = r2_score(y, y_pred)
        
        return self._standard_return(loss=loss, metric_per=metric_per, mode=mode, y=y, y_pred=y_pred)
        

In [7]:
trainer = Trainer(is_gpu=True, epoch=20, optimizer=optimizer, model=model, criterion=criterion)

Epoch:20, is GPU: True


In [8]:
config = {
    'b_m': 0.8,
    'm_p': 'D:\\Resource\\MDN\\MLPSelfAttention\\Test\\',
    'w_e': 5,
    # 'l_m': {
    #     's_l': [43, 50, 74, 134, 148, 157, 166, 180],
    #     'gamma': 0.7
    # }
}

In [9]:
result = trainer.fit(t_loader, v_loader, **config)


module.layer1.weight : torch.Size([256, 4])
module.layer1.bias : torch.Size([256])
module.layer2.weight : torch.Size([128, 256])
module.layer2.bias : torch.Size([128])
module.output.Q_layer.weight : torch.Size([64, 128])
module.output.Q_layer.bias : torch.Size([64])
module.output.K_layer.weight : torch.Size([64, 128])
module.output.K_layer.bias : torch.Size([64])
module.output.V_layer.weight : torch.Size([64, 128])
module.output.V_layer.bias : torch.Size([64])


  0%|          | 0/11025 [00:00<?, ?batch/s]E:\anaconda\envs\deeplearning\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Epoch 1 Validating: 100%|██████████| 1379/1379 [00:04<00:00, 331.39batch/s, loss=0.0000, loss_avg=0.0000, val_metric=0.9807]


Save Best model: Metric:0.9807, Loss Avg:0.0000


Epoch 2 Validating: 100%|██████████| 1379/1379 [00:04<00:00, 337.61batch/s, loss=0.0000, loss_avg=0.0000, val_metric=0.9827]


Save Best model: Metric:0.9827, Loss Avg:0.0000


Epoch 3 Validating: 100%|██████████| 1379/1379 [00:03<00:00, 345.88batch/s, loss=0.0000, loss_avg=0.0000, val_metric=0.9828]


Save Best model: Metric:0.9828, Loss Avg:0.0000


Epoch 6 Validating: 100%|██████████| 1379/1379 [00:04<00:00, 325.27batch/s, loss=0.0000, loss_avg=0.0000, val_metric=0.9828]


Save Best model: Metric:0.9828, Loss Avg:0.0000


Epoch 20 Validating: 100%|██████████| 1379/1379 [00:03<00:00, 347.98batch/s, loss=0.0000, loss_avg=0.0000, val_metric=0.9828]
